In [1]:
import librosa as lb
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
location = "C:/dnn_data/"
duration = 4
sampling_rate = 44100
#3*44100 = 259,40
#4*44100 = 345*30 = 10350
total_length = duration * sampling_rate
# 데이터 추출(chroma, mfcc)
def featureExtract(file_location, selection):
    #샘플링된 주파수 로드
    data, sr = lb.core.load(file_location, sr=sampling_rate, mono=True, res_type="kaiser_fast") 
    # 파일이 일정이상 커지면 그 뒤 값은 모두 랜덤
    if len(data)> total_length:
        max_offset = len(data) - total_length
        offset = np.random.randint(max_offset)
        data = data[offset:(total_length+offset)]
    else:
        if total_length > len(data):
            max_offset = total_length - len(data)
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, total_length - len(data) - offset), "constant")
    if selection==0:
        data = lb.feature.chroma_stft(data, sr=sampling_rate, n_chroma=30)
        data = np.reshape(data,10350)
    elif selection==1:
        data = lb.feature.mfcc(data, sr=sampling_rate, n_mfcc=30)
        data = np.reshape(data,10350)
    else:
        data = lb.feature.mfcc(data, sr=sampling_rate, n_mfcc=30)
    # mfcc = np.mean(lb.feature.mfcc(data, sr=sampling_rate n_mfcc=40).T,axis=0)
    # return chroma_stft,mfcc
    return data

#데이터 로드
train = pd.read_csv(location+"audio_train.csv")

#인덱스 설정(label에 각각 숫자부여)
label_index = pd.DataFrame(columns=("labeled_num","label"))
label_index["labeled_num"] = range(len(train["label"].unique()))
label_index["label"] = train['label'].unique()
train=pd.merge(train, label_index, how="left")

# 숫자와 파일이름 빼고 제거
train = train.drop(['label','manually_verified'],axis=1)
trainFname = train.values[:,0]

#chroma_data 초기화
rows=len(train)
#pd.Series(featureExtract(location+'audio_train/8343332e.wav')).append('1')
chroma_data = np.zeros((len(train),10351))
chroma_data.shape
mfcc_data = np.zeros((len(train),10351))


In [ ]:
# 데이터 전체 chroma-stft
i=0
for fname in trainFname:
    labeled_num=train.loc[train['fname']==fname,"labeled_num"]
    #indexed_extract=np.hstack([labeled_num,featureExtract(location+"/audio_train/"+fname,0)])
    indexed_extract2=np.hstack([labeled_num,featureExtract(location+"/audio_train/"+fname,1)])
    #chroma_data[i] = indexed_extract
    mfcc_data[i] = indexed_extract2
    i+=1
    if i%9==0:
        #print(i,'번째:',chroma_data[i])
        print(i,'번째:',chroma_data[i])
    #길이:10360 = 40*259

In [ ]:
# chroma_stft 피쳐링 거친 결과를 저장하기
#np.savetxt('choma_data.csv',chroma_data)
np.savetxt("c:/dnn_data/mfcc_data.csv",mfcc_data)

In [ ]:
# chroma_stft csv를 로드한다.
#chroma_data=np.loadtxt('C:/Users/stu/chroma_data.csv')
mfcc_data=np.loadtxt('mfcc_data.csv')

In [ ]:
# train, test 데이터 분리
#train_set, test_set = train_test_split(chroma_data, test_size=0.3)
train_set, test_set = train_test_split(mfcc_data, test_size=0.3)
trainData = train_set[:,1:]
testData = test_set[:,1:]
trainLabel=train_set[:,0]
testLabel=test_set[:,0]


In [ ]:
## Convolution layer
tf.reset_default_graph()
# hyper parameters
learning_rate = 0.0005
training_epochs = 300
batch_size = 300
steps_for_validate = 5

X = tf.placeholder(tf.float32,[None, 10350])
X_feature = tf.reshape(X,[-1,30,345,1])
Y = tf.placeholder(tf.int32, [None, 1], name="Y")
Y_onehot=tf.reshape(tf.one_hot(Y, 41), [-1, 41])
p_keep_conv = tf.placeholder(tf.float32, name="p_keep_conv")
p_keep_hidden = tf.placeholder(tf.float32, name="p_keep_hidden")

# L1 SoundIn shape=(?, 30, 345, 1)
# 30-2 = 28, 345-23 = 322
W1 = tf.get_variable("W1", shape=[3, 3, 1, 32],initializer=tf.contrib.layers.xavier_initializer())
L1 = tf.nn.conv2d(X_feature, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1],strides=[1, 3, 3, 1], padding='SAME') 
L1 = tf.nn.dropout(L1, p_keep_conv)

# L2 Input shape=(?,10,115,32)
W2 = tf.get_variable("W2", shape=[3, 3, 32, 64],initializer=tf.contrib.layers.xavier_initializer())
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 23, 1],strides=[1, 2, 23, 1], padding='SAME') 
L2 = tf.nn.dropout(L2, p_keep_conv)

# L3 Input shape=(?,2,23,64)
W3 = tf.get_variable("W3", shape=[3, 3, 64, 128],initializer=tf.contrib.layers.xavier_initializer())
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 5, 5, 1],strides=[1, 5, 5, 1], padding='SAME') 
L3 = tf.nn.dropout(L3, p_keep_conv)
L3_flat= tf.reshape(L3, shape=[-1, 128])

# Final FC 2*3*128 inputs -> 41 outputs
W4 = tf.get_variable("W4", shape=[128, 512],initializer=tf.contrib.layers.xavier_initializer())
L4 = tf.nn.relu(tf.matmul(L3_flat, W4))
L4 = tf.nn.dropout(L4, p_keep_hidden)
W_o = tf.get_variable("W_o", shape=[512,41],initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([41]))
logits = tf.matmul(L4, W_o) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels= Y_onehot))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam optimizer
predict_op = tf.argmax(logits, 1, name="pred")

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(trainData) / batch_size)
    for i in range(total_batch):
        batch_xs = trainData[i*batch_size:(i+1)*batch_size]
        batch_ys = trainLabel[i*batch_size:(i+1)*batch_size].reshape(-1, 1)
        feed_dict = {X: batch_xs, Y: batch_ys, p_keep_conv: .7, p_keep_hidden: 0.5}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    if epoch % steps_for_validate == steps_for_validate-1:
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_onehot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        x=np.random.choice(testLabel.shape[0], 500, replace=False)
        print('Accuracy:', sess.run(accuracy, feed_dict={
                X: testData[x], Y: testLabel[x].reshape(-1, 1), p_keep_conv: 1, p_keep_hidden: 1}))
        #save_path = saver.save(sess, 'c:/dnn_data/')
print('Finished!')

# chroma_data는 40%에서 고립